<a href="https://colab.research.google.com/github/callmemark/Correlation-of-Night-Air-Temperature-and-light-pollution-in-major-cities-in-the-Philippines/blob/main/Thesis_Proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Correlation of Night Air Temperature and light pollution in major cities in the Philippines </h1>


---
</br></br>




> Fernandez, Jaron Rix
</br>
Velmonte, Mark john A.









<h1>Preperation 1: Install Libraries</h1>

> Install neccesary libraries

In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install --upgrade pip --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install ipywidgets --quiet

# Install other libraries
!pip install splot --quiet
!pip install -U mgwr --quiet

#!pip freeze

<h1>Preperation 2: Import Libraries</h1>

> after installation we procced to import the libraries

In [11]:
#import libraries

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
from google.colab import drive
from os import path, getcwd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

<h1> Preperation 3: Moun Drive </h1>

> To access data from google drive, We need to give permission to mount the drive and gain access to folders and files

In [ ]:
#mount the google drive used
from google.colab import drive
drive.mount('/content/drive')

<h1>Preperation 4: File Reference </h1>

> Create a dictionary to use for file path reference and for process automation

In [12]:
GPKG_DATASET_DIRECTORY = "/content/drive/My Drive/COLLEGE_THESIS_FOLDER/GPKG_FILES/"

gpkg_files_dict = {
    "Manila_City" : "Manila_City.gpk",
    "Taguig_City" : "Taguig_City.gpk",
    "Pasig_City" : "Pasig_City.gpk",
    "Paranaque_City" : "Paranaque_City.gpk",
    "Makati_City" : "Makati_City.gpk",
    "LasPinas_City" : "LasPinas_City.gpk",
    "Muntinlupa_City" : "Muntinlupa_City.gpk",
    "Pasay_City" : "Pasay_City.gpk",
    "Malabon_City" : "Malabon_City.gpk",
    "Butuan_City" : "Butuan_City.gpk",
}



DATASET_DIRECTORY = "/content/drive/My Drive/COLLEGE_THESIS_FOLDER/DataSet/CSV_climactic_factors_dataset/"

CITY_NAME_2011_2020= [
    "Manila_City_2011-2020",
    "Taguig_City_2011-2020",
    "Pasig_City_2011-2020",
    "Paranaque_City_2011-2020",
    "Makati_City_2011-2020",
    "Las Pinas_City_2011-2020",
    "Muntinlupa_City_2011-2020",
    "Pasay_City_2011-2020",
    "Malabon_City_2011-2020",
    "Bututan_City_2011-2020"
]

CITY_NAME_2010_2010= [
    "Manila_City_2010-2010",
    "Taguig_City_2010-2010",
    "Pasig_City_2010-2010",
    "Paranaque_City_2010-2010",
    "Makati_City_2010-2010",
    "Las Pinas_City_2011-2020",
    "Muntinlupa_City_2010-2010",
    "Pasay_City_2010-2010",
    "Malabon_City_2010-2010",
    "Bututan_City_2010-2010"
]

<h1>Preperation 5: Helper Function / Classes</h1>

> We created functions and classes for process that we know we will repeat 
<br>
> For modularity we adapt object oriented programming paradigm


In [ ]:
# Helper functions and Classes
# from t-19 to t-4

def get_file_path(city_name_arg):
  _fstr = DATASET_DIRECTORY + city_name_arg + ".csv"
  return _fstr

def read_and_comb_dataset(ds_n1 = str, ds_n2 = str, row_skip = 13):
  df_n1 = filter_data(pd.read_csv(ds_n1, skiprows=row_skip))
  df_n2 = filter_data(pd.read_csv(ds_n2, skiprows=row_skip))

  df_concat = [df_n1, df_n2]
  return pd.concat(df_concat)


def filter_data(data_frame_arg):
  modif_df = data_frame_arg
  year_in_data = np.array(data_frame_arg["YEAR"])

  if 2011 in year_in_data and 2010 in year_in_data:
    modif_df = data_frame_arg.drop(data_frame_arg[data_frame_arg["YEAR"] == 2011].index)
    #print("Removed 2011", 2010 in year_in_data)

  elif 2010 in year_in_data and 2020 in year_in_data:
    modif_df = data_frame_arg.drop(data_frame_arg[data_frame_arg["YEAR"] == 2010].index)
    #print("Removed 2010")

  elif 2021 in year_in_data and 2020 in year_in_data:
    modif_df = data_frame_arg.drop(data_frame_arg[data_frame_arg["YEAR"] == 2021].index)
    #print("Removed 2021")
  
  return modif_df



class ProcessMMreg():
  def __init__(self, City_Data, y_val_arg = "T2M"):
    self.city_data = City_Data
    self.y_val = y_val_arg

    self.city_df = pd.DataFrame(self.city_data)

    self.x = self.city_df.drop(columns = self.y_val)
    self.y = self.city_df[self.y_val]

    self.lr_model = LinearRegression()

  def run_standard_proc(self):
    self.split_train_test_data()
    self.fit_model()
    self.predict_data()
    self.get_prediction_plot()

  def split_train_test_data(self):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size = 0.3, random_state = 0)
    
  def fit_model(self):
    self.lr_model.fit(self.X_train, self.y_train)
  
  def predict_data(self):
    self.y_pred_train = self.lr_model.predict(self.X_train)

  def get_prediction_plot(self):
    plt.scatter(self.y_train, self.y_pred_train)
    #plt.xticks(self.y_pred_train)
    #plt.yticks(self.y_train)
    plt.show()

<h1>Procedure 1: Prepare Data</h1>

In this Section we prepare the data by filtering and selecting data that is neccesary to the research

In [ ]:
# create a empty dictionary
nt_city_df_dict = {}

# loop through the list
for tf1, tf2 in zip(CITY_NAME_2010_2010, CITY_NAME_2011_2020):
  # creating str | expected result ex. Manila_City
  tf1_name = tf1.split("_")[0] + "_" + tf1.split("_")[1]
  tf2_name = tf2.split("_")[0] + "_" + tf2.split("_")[1]

  # check if combining data from the same city
  if tf1_name == tf2_name:
    dict_key = tf1_name
    # combine data from 2010-2020
    comb_df = read_and_comb_dataset(get_file_path(str(tf1)), get_file_path(str(tf2)))

    # filter data from t-19 to t-04
    nightime_data = comb_df.loc[(comb_df["HR"] <= 4) | (comb_df["HR"] >= 19)]

    # create new dictinary item and add new data
    nt_city_df_dict[dict_key] = nightime_data

  # raise custom error if data are from different city
  elif tf1_name != tf2_name:
    raise Exception("Error: City Names did not match | cannot proceed")

print(nt_city_df_dict.keys())

In [ ]:
# year 2011-2020
year_range = [i for i in range(2010, 2021)]
# climactic factors
df_params = ["PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"]

# initialize empty dictionary
cities_yparam_avg_val = {}

# loop through the cities 
for city_fname in CITY_NAME_2011_2020: # or CITY_NAME_2010_2010 || Same value
  city_dict_key = city_fname.split("_")[0] + "_" + city_fname.split("_")[1]
  cities_yparam_avg_val[city_dict_key] = {}
  
  # loop through the climatic factors
  for param_name in df_params:
    # initialize new empty list every iteration
    initial_param_avg_list = []

    # loop through the year
    for _year in year_range:
      # get the average value of the current iterated climatic factor
      param_avg_val = np.average(np.array(nt_city_df_dict[city_dict_key].loc[nt_city_df_dict[city_dict_key]["YEAR"] == _year][param_name]))
      # appent to empty list
      initial_param_avg_list.append(param_avg_val)
      # add new dictionary key and its value
      cities_yparam_avg_val[city_dict_key][param_name] = np.array(initial_param_avg_list)



print(cities_yparam_avg_val.keys())